# Cuaderno 8: Calendarización de trabajos (job scheduling)

$\newcommand{\card}[1]{\left| #1 \right|}$
$\newcommand{\tabulatedset}[1]{\left\{ #1 \right\}}$

Los problemas de calendarización de trabajos (*job scheduling*) consisten en determinar el orden óptimo en el que deben ejecutarse ciertas tareas, en una o más máquinas, para optimizar un proceso productivo.

Suponer que se tiene dado un conjunto de $n$ trabajos de confección $J:= \tabulatedset{1, \ldots, n}$, los cuales  deben ser procesados secuencialmente en una máquina hiladora. La máquina contiene un carrete en el cual pueden ser cargados simultáneamente $B$ hilos de diferentes colores, seleccionados de un conjunto $H$ de hilos disponibles. Cada trabajo $j \in J$  requiere para su ejecución de un subconjunto $H_j \subset H$ de hilos. Para que $j$
pueda ser procesado, todos los hilos de $H_j$ deben estar cargados en el carrete. Si esto no ocurre, la máquina debe ser detenida para cargar los hilos que hagan falta y, de ser necesario, descargar hilos que no sean requeridos con la finalidad de liberar espacio en el carrete. Asumimos que $\card{H_j} \leq B, \forall j \in J$.

Se desea determinar el orden en el que los trabajos deben ser procesados para minimizar el número de paradas requeridas de la máquina. Para especificar el ordenamiento, se asignará a cada trabajo $j \in J$ un turno $t \in T$, con $T:= \tabulatedset{1, \ldots, n}$. 

Emplearemos las siguientes variables de decisión binarias: 

* $x_{jt}$, $j \in J$, $t \in T$, que indican si el trabajo $j$ es procesado en el turno $t$;
* $y_{ht}$, $h \in H$, $t \in T$, que determinan si el hilo $h \in H$ está cargado en la máquina durante el turno $t$; 
* $z_t$, $t \in T, \, t \geq 2$, que registran si es necesario parar la máquina entre los turnos $t-1$ y $t$.

Los valores de $z_t$ se determinan a partir de los valores de $y_{ht}$ como se indica a continuación. Notar que la expresión es $\card{y_{ht} - y_{h,t-1}}$ es igual a 1 si y sólo si el hilo $h$ entra o sale del carrete de la máquina entre los turnos $t-1$ y $t$. Además, definiendo:
\begin{align*}
w_{ht} \in & \{0, 1 \} \, : \\
& w_{ht} \geq y_{ht} - y_{h,t-1}, \\
& w_{ht} \geq y_{h,t-1} - y_{ht},
\end{align*}
tenemos que $w_{ht} \geq \card{y_{ht} - y_{h,t-1}}$. Notar que $\sum_{h \in H} w_{ht}$ es forzosamente mayor que cero si *por lo menos un* hilo entra o sale del carrete entre los turnos $t-1$ y $t$, es decir, si es necesario parar la máquina entre estos turnos. Por lo tanto, si tomamos $z_{t} \in \{0, 1 \}$, con $z_t \geq \frac{\sum_{h \in H} w_{ht}}{\card{H}}$, se tiene que $z_t = 1$ si es necesario detener la máquina entre los turnos $t-1$ y $t$ para cambiar algún hilo.

Empleando estas ideas, el problema de calendarización de trabajos descrito arriba puede ser formulado como el siguiente modelo de programación lineal entera.  

\begin{align*}
\min &\sum_{t \in T \setminus \{1\}} z_t\\ 
& \mbox{s.r.}\\
&\sum_{j \in J} x_{jt} = 1, \quad \forall t \in T, \\
&\sum_{t \in T} x_{jt} = 1, \quad \forall j \in J, \\
&\sum_{h \in H} y_{ht} \leq B, \quad \forall t \in T, \\
& x_{jt} \leq y_{ht}, \quad \forall h \in H_j, \, j \in J, \, t \in T,\\
& w_{ht} \geq y_{ht} - y_{h, t-1}, \quad, \forall h \in H, t \in T \setminus \{1\},\\
& w_{ht} \geq y_{h, t-1} - y_{ht}, \quad, \forall h \in H, t \in T \setminus \{1\},\\
& \sum_{h \in H} w_{ht} \leq \card{H} z_t, \forall t \in T \setminus \{1\},\\
& x_{jt}, y_{ht} \in \{0, 1\}, \quad \forall j \in J, h \in H, t \in T,\\
& w_{ht}, z_t \in \{0, 1\}, \quad \forall h \in H, t \in T \setminus \{1\}.
\end{align*}

La función objetivo mide la cantidad de turnos $t \in T \setminus \{1\}$ donde $z_t=1$. De lo expuesto arriba, forzosamente esta cantidad es, por lo menos, la cantidad de paradas de la máquina dentro de la solución. Por otra parte, como se trata de un problema de minimización, puede demostrarse que, en toda solución óptima, el valor de la función objetivo coincide con el número de paradas de la máquina.

Las dos primeras de restricciones establecen condiciones de coherencia para que las variables $x_{jt}$ correspondan a un ordenamiento de los trabajos: cada trabajo debe ser asignado a un turno, y cada turno debe tener un trabajo asignado.

La tercera familia de restricciones indica que, en cada turno, la cantidad de hilos cargados en el carrete no puede exceder su capacidad.

La cuarta familia de restricciones establece que, para que un trabajo sea procesado, todos los hilos requeridos por el mismo deben estar cargados.

Las últimas tres familias de restricciones ajustan el valor de $z_t$ como se explicó arriba, con la finalidad de que $z_t = 1$ cada vez que es necesario parar la máquina entre los turnos $t-1$ y $t$.


Vamos a implementar este modelo usando la interfaz Python de Gurobi.


Definimos primero los conjuntos $J$, $H$, $T$; y los parámetros $H_j$ y $B$:

In [1]:
from gurobipy import *
# import random

# Conjuntos y parametros del modelo
# Familia HH de conjuntos de hilos requeridos por cada trabajo
HH = tupledict({1 : {2, 3, 5},
                2 : {1, 3, 4},
                3 : {1, 4, 5},
                4 : {1, 3},
                5 : {1, 2, 4},
                6 : {1, 4, 5},
                7 : {3, 5}})

# Cantidad de hilos que pueden estar en el carrete
B = 3

# ---- a partir de aqui, los parametros se calculan en base a los anteriores ----

# Conjunto de trabajos
J = HH.keys()

# Conjunto de turnos T y de turnos sin el primero
T = J
T2 = [i for i in T if i!=1]

# Conjunto de todos los hilos, calculado como la union de Hj para j in J
H = set()
for j in J:
    H = H | HH[j]

print("H = {}".format(H))
print("J = {}".format(J))
print("T = {}".format(T))
print("HH = {}".format(HH))


H = {1, 2, 3, 4, 5}
J = [1, 2, 3, 4, 5, 6, 7]
T = [1, 2, 3, 4, 5, 6, 7]
HH = {1: {2, 3, 5}, 2: {1, 3, 4}, 3: {1, 4, 5}, 4: {1, 3}, 5: {1, 2, 4}, 6: {1, 4, 5}, 7: {3, 5}}


Definimos ahora el objeto modelo y las variables del modelo:

In [2]:
m = Model('job-scheduling')

# Asignacion turnos a trabajos
x = m.addVars(J, T, vtype = GRB.BINARY, name="x")

# Estado de hilos en el carrete
y = m.addVars(H, T, vtype = GRB.BINARY, name="y")

# Paradas de la maquina
z = m.addVars(T2, vtype = GRB.BINARY, name="z")

# Auxiliar: cambio de los hilos en el carrete
w = m.addVars(H, T2, vtype = GRB.BINARY, name="w")

Set parameter Username
Academic license - for non-commercial use only - expires 2022-02-03


Definimos la función objetivo:

In [3]:
# minimizar paradas de la maquina
m.setObjective(z.sum('*'), GRB.MINIMIZE)

Finalmente, implementamos las restricciones del modelo. El método `addConstrs` nos permite agregar una familia completa de restricciones al modelo, empleando un generador.

1. A cada trabajo $j$ se le asigna un turno único y en cada turno se ejecuta exactamente un trabajo.

In [4]:
# A cada turno se le asigna exactamente un trabajo
m.addConstrs((x.sum('*', t) == 1 for t in T), name='turno_t') 

# A cada trabajo se le asigna exactamente un turno
m.addConstrs((x.sum(j, '*') == 1 for j in J), name='turno_j') 

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>}

2. En cada turno, la cantidad de hilos cargados en la máquina no supera la capacidad del carrete.

In [5]:
# Respetar capacidad del carrete
m.addConstrs((y.sum('*', t) <= B for t in T), name='carrete') 

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>}

3. Si el trabajo $j$ es procesado en el turno $t$, entonces todos los hilos requeridos para este trabajo deben estar cargados en el carrete en el turno $t$.

In [6]:
# Cargar todos los hilos requeridos para procesar un trabajo
m.addConstrs((x[j,t] <= y[h,t] for t in T for j in J for h in HH[j]), name='hilos') 

{(1, 1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7, 5): <gurobi.Constr *Awaiting Model Upd

4. Fijamos el valor de las variables auxiliares $w_{ht}$:

In [7]:
# Definir w[h,t] en función de y[h,t] y y[h,t-1]
m.addConstrs((w[h,t] >= y[h,t] - y[h,t-1] for h in H for t in T2), name='defw1') 
m.addConstrs((w[h,t] >= y[h,t-1] - y[h,t] for h in H for t in T2), name='defw2') 

{(1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 6): <gurobi.Constr *Awaiting Model Update*>,
 (2, 7): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 3): <gurobi.Constr *Awaiting Model Update*>,
 (3, 4): <gurobi.Constr *Awaiting Model Update*>,
 (3, 5): <gurobi.Constr *Awaiting Model Update*>,
 (3, 6): <gurobi.Constr *Awaiting Model Update*>,
 (3, 7): <gurobi.Constr *Awaiting Model Update*>,
 (4, 2): <gurobi.Constr *Awaiting Model Update*>,
 (4, 3): <gurobi.Constr *Awaiting Model Update*>,


5. Finalmente, fijamos el valor de $z_t$ de tal forma que $z_t=1$ cada vez que sea necesario para la máquina entre los turnos $t-1$ y $t$:

In [8]:
# Definir z[t] en función de w[h,t]
m.addConstrs((w.sum('*', t) <= len(H)*z[t] for t in T2), name='defz') 

{2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>}

Optimizamos el modelo:

In [9]:
m.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 220 rows, 120 columns and 615 nonzeros
Model fingerprint: 0x3d8ebb5a
Variable types: 0 continuous, 120 integer (120 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Found heuristic solution: objective 6.0000000
Presolve removed 104 rows and 30 columns
Presolve time: 0.02s
Presolved: 116 rows, 90 columns, 481 nonzeros
Variable types: 0 continuous, 90 integer (90 binary)
Found heuristic solution: objective 5.0000000

Root relaxation: objective 0.000000e+00, 122 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   68    5.00000    0.00000   100%  

Mostramos la solución: instalaciones a construir, asignación de clientes a instalaciones y clientes no atendidos.

In [10]:
# Mostrar solucion
print('Orden de procesamiento de trabajos:')
print([j for t in T for j in J if x[j,t].x >= 0.9])

print('Numero de paradas: {}'.format(m.objVal))

print('Estado del carrete:')
for t in T:
    print('{}: {}'.format(t, [h for h in H if y[h,t].x >= 0.9]))

Orden de procesamiento de trabajos:
[1, 7, 6, 3, 5, 2, 4]
Numero de paradas: 3.0
Estado del carrete:
1: [2, 3, 5]
2: [2, 3, 5]
3: [1, 4, 5]
4: [1, 4, 5]
5: [1, 2, 4]
6: [1, 3, 4]
7: [1, 3, 4]


## Bonus extra
### Fijando brechas de optimalidad y tiempos límites de ejecución

Cuando los modelos a optimizar son grandes y demandan demasiado tiempo de ejecución, puede ser útil especificar un tiempo límite de ejecución para el solver o un rango de tolerancia para la brecha de optimalidad (de manera que la optimización termine tan pronto se encuentre una solución factible cuya brecha de optimalidad se encuentre dentro de ese rango).

En Gurobi esto se consigue fijando el valor de ciertos *parámetros* del solver. El tiempo máximo de ejecución está determinado por el [parámetro `TimeLimit`](https://www.gurobi.com/documentation/9.0/refman/timelimit.html), mientras que el rango de tolerancia para la brecha de optimalidad se especifica a través del [parámetro `MIPGap`](https://www.gurobi.com/documentation/9.0/refman/mipgap2.html)

Podemos asignar valores a los parámetros ajustando la propiedad `Params` en el objeto modelo.  

In [ ]:
# Fijar tiempo máximo de ejecución en 60 segundos
m.Params.TimeLimit = 60

# Fijar brecha de optimalidad aceptable en 10%
m.Params.MIPGap = 0.1

## Código completo

Reproducimos a continuación el código completo del ejemplo:

In [12]:
# Curso de implementación de programas lineales enteros
# Ejemplo: Modelo de job scheduling
# EPN (2020)

from gurobipy import *
import random as rd

try:
    # Conjuntos y parametros del modelo
    # Familia HH de conjuntos de hilos requeridos por cada trabajo
    # HH = tupledict({1 : {2, 3, 5},
    #                  2 : {1, 3, 4},
    #                  3 : {1, 4, 5},
    #                  4 : {1, 3},
    #                  5 : {1, 2, 4},
    #                  6 : {1, 4, 5},
    #                  7 : {3, 5}})
    HH = tupledict({j+1 : set([rd.randint(1,10) for i in range(3)]) for j in range(100)})

    # HH = {j : set([random.randint(1,5) for i in range(3)]) for j in range(1,101)}
    print("HH= {}".format(HH))

    # Cantidad de hilos que pueden estar en el carrete
    B = 3

    # Conjunto de trabajos
    J = HH.keys()

    # Conjunto de turnos T y de turnos sin el primero
    T = J
    T2 = [i for i in T if i!=1]

    # Conjunto de todos los hilos, calculado como la union de Hj para j in J
    H = set()
    for j in J:
        H = H | HH[j]
    
    # Crear el objeto modelo
    m = Model('job-scheduling')

    # Definir variables
    # Asignacion turnos a trabajos
    x = m.addVars(J, T, vtype = GRB.BINARY, name="x")

    # Estado de hilos en el carrete
    y = m.addVars(H, T, vtype = GRB.BINARY, name="y")

    # Paradas de la maquina
    z = m.addVars(T2, vtype = GRB.BINARY, name="z")

    # Auxiliar: cambio de los hilos en el carrete
    w = m.addVars(H, T2, vtype = GRB.BINARY, name="z")
    
    # Funcion objetivo: minimizar paradas de la maquina
    m.setObjective(z.sum('*'), GRB.MINIMIZE)

    # Restricciones
    # A cada turno se le asigna exactamente un trabajo
    m.addConstrs((x.sum('*', t) == 1 for t in T), name='turno_t') 

    # A cada trabajo se le asigna exactamente un turno
    m.addConstrs((x.sum(j, '*') == 1 for j in J), name='turno_j') 
    
    # Respetar capacidad del carrete
    m.addConstrs((y.sum('*', t) <= B for t in T), name='carrete') 
    
    # Cargar todos los hilos requeridos para procesar un trabajo
    m.addConstrs((x[j,t] <= y[h,t] for t in T for j in J for h in HH[j]), name='hilos') 
    
    # Definir w[h,t] en función de y[h,t] y y[h,t-1]
    m.addConstrs((w[h,t] >= y[h,t] - y[h,t-1] for h in H for t in T2), name='defw1') 
    m.addConstrs((w[h,t] >= y[h,t-1] - y[h,t] for h in H for t in T2), name='defw2') 
    
    # Definir z[t] en función de w[h,t]
    m.addConstrs((w.sum('*', t) <= len(H)*z[t] for t in T2), name='defz') 
    
    # Fijar tiempo máximo de ejecución en 30 segundos
    m.Params.TimeLimit = 90

    # Fijar brecha de optimalidad aceptable en 10%
    # m.Params.MIPGap = 0.1

    # Resolver el modelo
    m.optimize()
    
    # Mostrar solucion
    print('Orden de procesamiento de trabajos:')
    print([j for t in T for j in J if x[j,t].x >= 0.9])

    print('Numero de paradas: {}'.format(m.objVal))

    print('Estado del carrete:')
    for t in T:
        print('{}: {}'.format(t, [h for h in H if y[h,t].x >= 0.9]))
        
except GurobiError as e:
    print('Se produjo un error de Gurobi: codigo: ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Se produjo un error de atributo')

HH= {1: {8, 4}, 2: {1, 7}, 3: {2, 3}, 4: {9, 2}, 5: {3, 5, 6}, 6: {9, 2, 6}, 7: {2, 5, 6}, 8: {1, 2, 7}, 9: {10, 3, 7}, 10: {10, 6, 7}, 11: {4, 5, 6}, 12: {1, 2, 6}, 13: {3, 4, 5}, 14: {1, 3, 4}, 15: {8, 10, 6}, 16: {4, 5}, 17: {2, 10, 5}, 18: {1, 10, 3}, 19: {2, 10}, 20: {8, 10, 4}, 21: {8, 1, 6}, 22: {2, 6}, 23: {8, 10, 3}, 24: {9, 10, 7}, 25: {10, 5, 7}, 26: {9, 10, 3}, 27: {5, 6, 7}, 28: {5, 7}, 29: {9, 3, 1}, 30: {4, 5}, 31: {8, 10, 4}, 32: {8, 3, 7}, 33: {9, 4, 7}, 34: {9, 5, 6}, 35: {2, 3, 5}, 36: {10, 3, 7}, 37: {1, 10, 6}, 38: {1, 10, 3}, 39: {2, 10, 4}, 40: {9}, 41: {1, 3, 6}, 42: {2, 5}, 43: {10, 4, 5}, 44: {8, 5, 6}, 45: {8, 9, 6}, 46: {1, 5, 6}, 47: {8}, 48: {5, 6}, 49: {8, 9, 7}, 50: {10, 2}, 51: {10, 4, 7}, 52: {8, 9, 1}, 53: {8, 10, 2}, 54: {8, 4, 7}, 55: {9, 5, 7}, 56: {8, 1, 5}, 57: {6, 7}, 58: {8, 4, 6}, 59: {10, 6}, 60: {10, 6}, 61: {1, 5}, 62: {8, 1, 5}, 63: {9, 3, 6}, 64: {4, 5}, 65: {1, 2, 3}, 66: {9, 2}, 67: {8, 1, 5}, 68: {8, 5, 6}, 69: {8, 3, 5}, 70: {2, 3, 5}